In [1]:
# !pip install antspyx
# !pip install SimpleITK
# !pip install tqdm

In [1]:
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from preprocess import Preprocess
from runtime import RunTime

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-tgm4yfpf because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Quadro RTX 8000, compute capability 7.5


In [2]:
data_dict = {'raw_data_dir': '/tf/data/brats_2020/raw/train',
             'processed_data_dir': '/tf/data/brats_2020/processed/norm-surface-loss',
             'base_model_name': 'brats_bl',
             'model_dir': '/tf/data/brats_2020/models/boundary-loss',
             'prediction_dir': '/tf/data/brats_2020/predictions/norm-surface-loss/boundary-loss',
             'raw_paths_csv': '/tf/github/norm-surface-loss/paths.csv',
             'modality': 'mr',
             'mask': ['_seg.nii.gz'], 
             'images': {'t1': ['t1.nii.gz'], 't2': ['t2.nii.gz'], 'tc': ['t1ce.nii.gz'], 'fl': ['flair.nii.gz']}, 
             'labels': [0, 1, 2, 4],
             'augment': True,
             'cache_size': 10,
             'cache_replacement_rate': 0.25,
             'batch_size': 2,
             'patch_size': [128, 128, 128],
             'loss': 'dice',
             'epochs': 150,
             'train_proto': 'crossval'}

# data_dict = {'raw_data_dir': '/tf/data/resection_cavity/processed/nifti_qa/',
#              'processed_data_dir': '/tf/data/resection_cavity/processed/tfrecord/',
#              'base_model_name': 'rc_model',
#              'model_dir': '/tf/data/resection_cavity/models/',
#              'prediction_dir': '/tf/data/resection_cavity/predictions/',
#              'raw_paths_csv': '/tf/github/NecrosisRecurrence/resection_cavity/paths.csv',
#              'mask': ['truth.nii.gz', 'mask'], 
#              'images': {'t1': ['t1.nii.gz', 'POSTOP_T1_W_SN41mm.nii.gz'], 
#                         't2': ['t2.nii.gz', 'POSTOP_AxT2_W_SN41mm.nii.gz'], 
#                         'tc': ['tc.nii.gz', 'POSTOP_T13D_W_SN41mm.nii.gz'], 
#                         'fl': ['fl.nii.gz', 'FLAIR_W_SN41mm.nii.gz']}, 
#              'labels': [0, 1], 
#              'augment': True,
#              'cache_size': 15,
#              'cache_replacement_rate': 0.25,
#              'batch_size': 2,
#              'patch_size': (128, 128, 128),
#              'epochs': 100,
#              'train_proto': 'crossval'}
base_dir = os.path.abspath('/tf/github/norm-surface-loss')
json_file = os.path.join(base_dir, 'boundary-loss-input.json')
with open(json_file, 'w') as outfile: 
    json.dump(data_dict, outfile)

In [3]:
preprocessor = Preprocess(json_file)
#preprocessor.run()

In [4]:
train = RunTime(json_file)
train.run()

Starting split 1/5
Epoch 1/150
alpha = 1.0
250/250 [==============================] - 100s 322ms/step - loss: 0.7273
INFO:tensorflow:Assets written to: /tf/data/brats_2020/models/boundary-loss/brats_bl_current_model_split_1/assets
Val loss IMPROVED from inf to 0.9565773606300354
INFO:tensorflow:Assets written to: /tf/data/brats_2020/models/boundary-loss/brats_bl_best_model_split_1/assets
Epoch 2/150
alpha = 0.9933333333333333
250/250 [==============================] - 94s 319ms/step - loss: 0.3860
INFO:tensorflow:Assets written to: /tf/data/brats_2020/models/boundary-loss/brats_bl_current_model_split_1/assets
Val loss IMPROVED from 0.9565773606300354 to 0.7779048681259155
INFO:tensorflow:Assets written to: /tf/data/brats_2020/models/boundary-loss/brats_bl_best_model_split_1/assets
Epoch 3/150
alpha = 0.9866666666666667
250/250 [==============================] - 94s 318ms/step - loss: 0.3622
INFO:tensorflow:Assets written to: /tf/data/brats_2020/models/boundary-loss/brats_bl_current_mod

In [2]:
import ants
import numpy as np
import SimpleITK as sitk

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-gxsbt_xk because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
x = ants.image_read('/tf/remove_later/ki67-docker-cases/10-18-2021/tc.nii.gz')

ki67 = sitk.ReadImage('/tf/remove_later/ki67-docker-cases/10-18-2021/CD.nii.gz')
ki67 = sitk.GetArrayFromImage(ki67).T
ki67 = x.new_image_like(data = ki67)
overlay = ants.image_read('/tf/remove_later/ki67-docker-cases/10-18-2021/brainmask.nii.gz')
rc = ants.image_read('/tf/remove_later/ki67-docker-cases/10-18-2021/rcmask.nii.gz')

ki67 = ki67.numpy()
overlay = overlay.numpy()
rc = rc.numpy()
overlay -= rc
overlay *= ki67
overlay = x.new_image_like(data = overlay)

ants.plot(x, 
          overlay = overlay, 
          crop = True, 
          nslices = 30, 
          ncol = 6, 
          overlay_alpha = 1.0, 
          cbar = True, 
          title = 'CD Map', 
          filename = '/tf/remove_later/ki67-docker-cases/10-18-2021/cd.png', 
          dpi = 500, 
          overlay_cmap = 'seismic', 
          black_bg = True, 
          axis = 2)

In [70]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, metrics, mixed_precision
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K

################### Loss functions for custom objects ###################
def dice_loss_weighted(y_true, y_pred):
    smooth = 0.0000001

    # (batch size, depth, height, width, channels)
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_true.shape) - 1))
    
    y_true = y_true[..., 0:2]

    Wk = K.sum(y_true, axis = axes)
    Wk = 1. / (K.square(Wk) + 1)

    num = K.sum(K.square(y_true - y_pred), axis = axes)
    den = K.sum(K.square(y_true), axis = axes) + K.sum(K.square(y_pred), axis = axes) + smooth

    return K.sum(Wk * num, axis = -1) / K.sum(Wk * den, axis = -1)

def surface_loss(y_true, y_pred):    
    num_classes = 2
    dtm = y_true[..., num_classes:(2 * num_classes)]
    return K.mean(dtm * y_pred)

def dice_norm_surf_loss_wrapper(alpha):
    def dice_norm_surf_loss(y_true, y_pred):
        return dice_loss_weighted(y_true, y_pred) + (1.0 - alpha) * surface_loss(y_true, y_pred)
    return dice_norm_surf_loss

custom_objects = {'dice_loss_weighted': dice_loss_weighted, 
                  'dice_norm_surf_loss': dice_norm_surf_loss_wrapper(tf.Variable(1.0))}

In [12]:
base_dir = os.path.abspath('/tf/github/NecrosisRecurrence/ki67_docker/tag-hdbet/rcmodels/')
models = os.listdir(base_dir)
for model in models:
    path = os.path.join(base_dir, model)
    new_model_path = os.path.join(base_dir, '{}.h5'.format(model))
    model = load_model(path, custom_objects = custom_objects)
    tf.keras.models.save_model(model, new_model_path)

In [14]:
model = load_model(os.path.join(base_dir, 'rc_model_best_model_split_3.h5'), custom_objects = custom_objects)

In [15]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 12 0                                            
__________________________________________________________________________________________________
conv3d_30 (Conv3D)              (None, 128, 128, 128 1744        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_28 (BatchNo (None, 128, 128, 128 64          conv3d_30[0][0]                  
__________________________________________________________________________________________________
activation_28 (Activation)      (None, 128, 128, 128 0           batch_normalization_28[0][0]     
____________________________________________________________________________________________

In [19]:
weights, _ = model.layers[1].get_weights()
weights.shape

(3, 3, 3, 4, 16)